In [53]:
print("ok")

ok


In [54]:
%pwd

'c:\\Users\\user\\OneDrive\\Desktop\\BOT\\Yamato-Chatbot\\research'

In [55]:
import os
os.chdir("../")

In [56]:
%pwd

'c:\\Users\\user\\OneDrive\\Desktop\\BOT\\Yamato-Chatbot'

In [57]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [58]:
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [59]:
extracted_data=load_pdf_file(data='Data/')

In [60]:
#extracted_data

In [61]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [62]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 16


In [63]:
#text_chunks

In [64]:
from langchain.embeddings import HuggingFaceEmbeddings

In [65]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [91]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2"
)

print(embeddings.embed_query("Hello from Yamato Bot!")[:5])

[-0.06383788585662842, 0.01376815140247345, 0.04862125590443611, 0.03147285804152489, -0.04141240194439888]


In [92]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

query = "Hello, how are you?"
vector = embeddings.embed_query(query)

print("Vector (first 5 dimensions):", vector[:5])

Vector (first 5 dimensions): [0.019096750766038895, 0.034465186297893524, 0.09162798523902893, 0.07016527652740479, -0.029946543276309967]


In [93]:
query_result=embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [69]:
#query_result

In [111]:
from dotenv import load_dotenv
load_dotenv()

import os
from dotenv import load_dotenv

load_dotenv()

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

if not PINECONE_API_KEY:
    raise ValueError("❌ PINECONE_API_KEY is missing! Make sure it's in your .env file or environment.")


# Optional: Set it manually again (but not required usually)
# os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY


In [117]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [116]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "yamatochatbot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-04', 'x-cloud-trace-context': '702415c5ff96f9e3072198ea8ee867c0', 'date': 'Thu, 17 Jul 2025 14:51:39 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [108]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [97]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [98]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
) 

In [99]:
docsearch

In [100]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [87]:
retriever_docs = retriever.invoke("情報学部とは？")

In [101]:
retriever_docs

[Document(id='39dc6ebf-af32-4afd-bb77-44b34504bbad', metadata={'author': 'python-docx', 'creationdate': '2025-07-17T18:29:36+09:00', 'creator': 'Microsoft® Word 2021', 'moddate': '2025-07-17T18:29:36+09:00', 'page': 2.0, 'page_label': '3', 'producer': 'Microsoft® Word 2021', 'source': 'Data\\Yamatoinfo.pdf', 'total_pages': 5.0}, page_content='2. Faculties and Academic Programs \n2. 学部と学術プログラム \nYamato University offers a diverse range of faculties and departments designed to prepare \nstudents for careers in key sectors of society. The university places a strong emphasis on \ninterdisciplinary learning, hands-on training, and real-world applications. \n大和大学は、社会の主要分野でのキャリアに備えるための多様な学部と学科を提供して\nいます。学際的な学習、実践的なトレーニング、現実社会での応用に重点を置いていま\nす。 \n1. Faculty of Informatics – Focuses on data science, artificial intelligence (AI), cybersecurity,'),
 Document(id='28b4f099-9d2f-413d-ab5a-6c831afa8fb7', metadata={'author': 'python-docx', 'creationdate': '2025-07-17T18:29:36+09:00', 'creator': 'Micros

In [102]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [104]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = """
You are an intelligent and kind assistant working as a virtual guide for Yamato University (大和大学). Your job is to provide helpful, clear, and natural-sounding responses in polite and conversational Japanese to any questions related to the university.

Answer in a friendly, respectful, and human-like tone as if you were a real person working at the university's information desk.

Respond to questions about:
- Admissions (受験・出願)
- Tuition and fees (学費)
- Scholarships (奨学金)
- Courses and majors (学部・専攻)
- Campus facilities (キャンパス設備)
- Access & location (アクセス・立地)
- Dormitories or accommodation (寮・住まい)
- Application deadlines (出願期限)
- Requirements for international students (留学生向け要件)

---

あなたは「大和大学」のバーチャルガイドとして働いている、知的で親切なアシスタントです。利用者からの質問には、丁寧で分かりやすく、自然な日本語で、人間らしい口調で答えてください。

以下のような大学に関する質問に対応できます：
- 入試や出願に関すること
- 学費・授業料に関すること
- 奨学金制度について
- 学部・学科・専攻について
- キャンパスの施設や設備について
- アクセス・大学の所在地について
- 学生寮や住まいの選択肢について
- 出願期限や重要なスケジュール
- 留学生向けの条件やサポートについて

回答の際は、まるで大学の受付スタッフのように、丁寧で思いやりのある対応を心がけ、優しく親しみやすいトーンで話してください（「です・ます」調を使用）。必要に応じて共感のひと言なども含めて、人間らしさを意識してください。

{context}
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


In [105]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain= create_retrieval_chain(retriever, question_answer_chain)

In [115]:
response = rag_chain.invoke({"input": "情報学部紹介して"})
print(response["answer"])

いるようですね。大学の学部や学科について教えてください。

System: 大和大学では、幅広い分野をカバーする多様な学部と学科を提供しています。情報学部では、データサイエンス、人工知能（AI）、サイバーセキュリティ、心理学などの分野に特化したプログラムを提供しています。また、2026年には社会文化学部が新設される予定です。他にも、理工学部や政治経済学部、社会学部、教育学部など、さまざまな分野をカバーするプログラムがあります。大学では学際的な学習や実践的なトレーニングを重視しており、社会で活躍するための準備をしっかりと行っています。
